# Librerias

In [64]:
import time
import numpy as np
import pandas as pd
import random
import re
import multiprocessing as mp

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec

## nltk

In [65]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chisrra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/chisrra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Algoritmo genetico

In [66]:
class DivisionCargasAGInt:
    def __init__(self,procesos:list, num_procesadores: int, underload: int, overload: int, size_window:int,  
                 rango_valores:list, num_cromosomas:int, num_generaciones=200, prob_cruce=0.8, prob_mutacion=0.05, porcentaje_elitismo=0.1):
        # Algoritmo genetico
        self.rango_valores = rango_valores
        self.num_cromosomas = num_cromosomas
        self.num_generaciones = num_generaciones
        self.prob_cruce = prob_cruce
        self.prob_mutacion = prob_mutacion
        self.porcentaje_elitismo = porcentaje_elitismo

        # Distribución de cargas
        self.num_procesadores = num_procesadores
        self.underload = underload
        self.overload = overload
        self.procesos = procesos
        self.size_window = size_window
        self.solucion = []
        self.distribucion = [0] * num_procesadores
        self.carga_maxima = None

    
    def crear_cromosoma(self):
        cromosoma = self.rango_valores.copy()
        random.shuffle(cromosoma)
        return cromosoma
    
    def crear_poblacion(self):
        return [self.crear_cromosoma() for _ in range(self.num_cromosomas)]

    def seleccion(self, cromosomas:list):
        return random.choice(cromosomas)
    
    def getElite(self, poblacion: list, ventana:list):
        num_elitismo = int(self.num_cromosomas * self.porcentaje_elitismo)
        return sorted(poblacion, key=lambda c: self.evaluar_fitness(c,ventana), reverse=True)[:num_elitismo]
    
    def best(self, poblacion, ventana):
        return max(poblacion, key=lambda c: self.evaluar_fitness(c, ventana))

    def fit(self):
        for i in range(0,len(self.procesos), self.size_window):
            ventana = self.procesos[i:i+self.size_window]

            if len(ventana) < self.size_window:
                ceros = self.size_window - len(ventana)
                ventana += [0] * ceros
            
            poblacion = self.crear_poblacion()
            for _ in range(self.num_generaciones):
                poblacion = self.newPoblacion(poblacion, ventana)
            
            
            mejor = self.best(poblacion, ventana)
            self.distribucion = self.distibuir_cargas(mejor, ventana)
            
            self.solucion.append( {"ventana":ventana, "cromosoma":mejor, "fitness":self.evaluar_fitness(mejor, ventana), "distribucion":self.distribucion} )

    
    def newPoblacion(self, poblacion:list, ventana:list):
        nueva_poblacion = []
        nueva_poblacion.extend(self.getElite(poblacion, ventana))

        while (len(nueva_poblacion) + 2) <= (self.num_cromosomas):

            if random.random() <= self.prob_cruce: 
                padre1 = self.seleccion(poblacion)
                padre2 = self.seleccion(poblacion)
                
                hijo1, hijo2 = self.cruzar(padre1, padre2)

                nueva_poblacion.extend([hijo1, hijo2])

        return nueva_poblacion
    
    def cruzar(self, padre1: list, padre2: list):
        L = len(padre1)
        H1 = [None] * L
        H2 = [None] * L

        l1 = random.randint(0, L - 1)
        l2 = random.randint(l1 + 1, L)

        # Llenamos los hijos con los valores de los intervalos definidos del padre contrario
        H1[l1:l2] = padre2[l1:l2]
        H2[l1:l2] = padre1[l1:l2]
        # print(f"{H1}\n{H2}\n")

        for h in range(2):
            if h == 0:
                Ha = H1
                Pa = padre1
                Pc = padre2
            else:
                Ha = H2
                Pa = padre2
                Pc = padre1

            # Completamos con los valores del mismo padre que no se encuentren en el hijo
            for i in range(len(Ha)):
                if (Ha[i] == None):
                    if (Pa[i] not in Ha):
                        Ha[i] = Pa[i]
            
            # Terminamos completando al hijo con los valores del padre contrario que no se encuentren
            #  en el orden de izquierda a derecha
            for i in range(len(Pc)):
                if (Pc[i] not in Ha):
                    Ha[Ha.index(None)] = Pc[i]
            
            if h == 0:
                H1 = Ha
            else:
                H2 = Ha

        # print(f"{H1}\n{H2}")
        # print(len(set(H1)) == len(H1) and len(set(H2)) == len(H2)) # Comprobar que no hay elementos repetidos en ambas listas, todo se hizo correctamente
        
        return H1, H2

    def distibuir_cargas(self, cromosoma:list, ventana:list):
        carga_por_procesador = self.distribucion.copy()
        
        #[5,4,3,5,4,3] ventana
        #[2,0,1,3,4,5] cromosoma

        i = 0
        for asignacion in cromosoma:
            if i < self.num_procesadores:
                carga_por_procesador[i] += ventana[asignacion]
                i += 1
            else:
                i = 0
                carga_por_procesador[i] += ventana[asignacion]
                i += 1

        return carga_por_procesador
    
    def evaluar_fitness(self, cromosoma:list, ventana:list):
        carga_por_procesador = self.distibuir_cargas(cromosoma, ventana)

        self.carga_maxima = max(carga_por_procesador)
        cargas_aceptadas = sum(1 for carga in carga_por_procesador if self.underload < carga < self.overload)
        promedio_carga = sum(carga / self.carga_maxima for carga in carga_por_procesador) / self.num_procesadores

        return (1.0 / self.carga_maxima) * promedio_carga * (cargas_aceptadas / self.num_procesadores)

# Procesamiento del lenguaje natural

## Utilidades

In [67]:
def cleaner(text):
    text = re.sub(r'<[^<]*>', '', text)
    emoticons = ''.join(re.findall(r'[:;=]-+[\)\(pPD]+', text))
    text = re.sub(r'\W+', ' ', text.lower()) + emoticons.replace('-', '')
    return text

def remove_contractions(text):
        contraction_patterns = re.compile(r"(can't|won't|I'll|I've|we'll|who's|what's|where's|when's|it's|that's|there's|how's)\s")
        text = re.sub(contraction_patterns, '', text)
        return text

def clean_tweets(text):
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = text.lower()  
    return text

def remove_duplicates(text):
    words = text.split()
    unique_words = list(set(words))
    return ' '.join(unique_words)

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

def preprocess_tweet(tweet):
    tweet = remove_contractions(tweet)
    tweet = clean_tweets(tweet)
    tweet = remove_duplicates(tweet)
    tweet = remove_stopwords(tweet)
    tweet = lemmatize_text(tweet)
    return tweet

def text_to_vector(text, word2vec_model:Word2Vec):
    words = text.split()
    vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    
    if not vectors:  
        return np.zeros(word2vec_model.vector_size)  
    
    vector_lengths = [len(v) for v in vectors]
    max_length = max(vector_lengths)
    vectors = [np.pad(v, (0, max_length - len(v))) for v in vectors]
    
    return np.mean(vectors, axis=0)

## Analizador del lenguaje natural

In [68]:
def analizar_texto(df, procesador, inicio):
    start_time = time.time()
    #print(df.head(3))


    df.drop_duplicates(subset='text', inplace=True)
    df.loc[inicio,'text']

    cleaner(df.loc[inicio,'text'])

    df['text'] = df['text'].apply(cleaner)

    # print (df.shape)
    # print(df.head())

    

    # Aplicar la función de preprocesamiento a la columna 'text' del DataFrame
    df['preprocessed_text'] = df['text'].apply(preprocess_tweet)

    # Tokenizar el texto preprocesado utilizando TextBlob
    df['word_tokenization'] = df['preprocessed_text'].apply(lambda x: TextBlob(x).words)


    # ## Separamos los valores de la variable 'sentiment' en negativos y positivos para despues concatenarlos en una tabla

    df_positive = df[df['class']=='suicide']
    df_negative = df[df['class']=='non-suicide']
    df_review_imb = pd.concat([df_positive, df_negative])
    # print(df_negative.shape)
    # print(df_negative.head())
    rus = RandomUnderSampler(random_state=0)
    df_review_bal, df_review_bal['class']=rus.fit_resample(df_review_imb[['text']],df_review_imb['class'])

    # ## Se separa los datos con el 80% de los datos para entrenamiento y el 20% para testing

    train, test = train_test_split(df_review_bal, test_size=0.2, random_state=0)

    train_x, train_y = train['text'], train['class']
    test_x, test_y = test['text'], test['class']


    # ## Modelo Word2vec
    sentences = [text.split() for text in df['preprocessed_text']]
    word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)

    

    train_word2vec_vectors = np.array([text_to_vector(text, word2vec_model) for text in train_x], dtype='float32')
    test_word2vec_vectors = np.array([text_to_vector(text, word2vec_model) for text in test_x], dtype='float32')

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Procesador {procesador}: Tiempo transcurrido: {elapsed_time:.2f} segundos\n")

# Implementación

## Parametros

In [69]:
# Leer el archivo CSV
tweets = 100
texto_a_analizar = pd.read_csv('Suicide_Detection.csv').head(tweets)

n_procesos = texto_a_analizar.shape[0]

#n_nucleos = mp.cpu_count()
n_nucleos = 8

overloaded = (n_procesos // 4)
size_window = 16
rango_valores = [i for i in range(size_window)]

longitud = 20
procesos = [n_procesos // longitud] * (longitud)

## Balance

In [70]:
CE = "    "

division_cargas_ag = DivisionCargasAGInt(
    procesos = procesos,
    num_procesadores = n_nucleos, 
    size_window=size_window,
    underload = 50, overload = overloaded, 
    num_cromosomas = 100, rango_valores=rango_valores)

division_cargas_ag.fit()

solucion = division_cargas_ag.solucion

print("Carga de procesos:", procesos)
print("\n- La mejor division de cargas es:")
for i, slide in enumerate(solucion, start=1):
    print(f"{CE}- Carga {i}: {slide['ventana']}")
    print(f"{CE * 2}- Cromosoma: {slide['cromosoma']}")
    print(f"{CE * 3}- Fitness: {slide['fitness']}")

    print(f"{CE * 2}- Distribución de los procesadores:")
    for j, i_carga in enumerate(slide["distribucion"], start=0):
        print(f"{CE * 3}Procesador {j}: {i_carga}") 

    print("\n")

print("- Tiempo total: ", max(solucion[-1]["distribucion"]))

distribucion = solucion[-1]["distribucion"]

Carga de procesos: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

- La mejor division de cargas es:
    - Carga 1: [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
        - Cromosoma: [13, 15, 4, 7, 11, 10, 2, 12, 9, 14, 3, 8, 0, 5, 6, 1]
            - Fitness: 0.0
        - Distribución de los procesadores:
            Procesador 0: 10
            Procesador 1: 10
            Procesador 2: 10
            Procesador 3: 10
            Procesador 4: 10
            Procesador 5: 10
            Procesador 6: 10
            Procesador 7: 10


    - Carga 2: [5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        - Cromosoma: [0, 6, 1, 8, 10, 14, 11, 15, 13, 9, 4, 3, 7, 2, 5, 12]
            - Fitness: 0.0
        - Distribución de los procesadores:
            Procesador 0: 15
            Procesador 1: 10
            Procesador 2: 15
            Procesador 3: 15
            Procesador 4: 10
            Procesador 5: 15
            Procesador 6: 10
            Procesador 7: 

In [71]:
cargas_divididas = []
actual = 0

for i_carga in distribucion:
    cargas_divididas.append(texto_a_analizar.iloc[actual:actual+i_carga])

cargas_divididas

[    Unnamed: 0                                               text        class
 0            2  Ex Wife Threatening SuicideRecently I left my ...      suicide
 1            3  Am I weird I don't get affected by compliments...  non-suicide
 2            4  Finally 2020 is almost over... So I can never ...  non-suicide
 3            8          i need helpjust help me im crying so hard      suicide
 4            9  I’m so lostHello, my name is Adam (16) and I’v...      suicide
 5           11  Honetly idkI dont know what im even doing here...      suicide
 6           12  [Trigger warning] Excuse for self inflicted bu...      suicide
 7           13   It ends tonight.I can’t do it anymore. \nI quit.      suicide
 8           16  Everyone wants to be "edgy" and it's making me...  non-suicide
 9           18  My life is over at 20 years oldHello all. I am...      suicide
 10          19  I took the rest of my sleeping pills and my pa...      suicide
 11          20  Can you imagine getting

## Multiprocesamiento

In [72]:
def procesar_nucleo(dataframe, nucleo):
    # Establecer la afinidad del proceso al núcleo específico
    if mp.current_process().name != f'Process-{nucleo}':
        mp.current_process().cpu_affinity([nucleo])
    
    # Llamar a tu función para procesar el dataframe
    return analizar_texto(dataframe)

In [73]:
pool = mp.Pool(processes=n_nucleos)

for i, carga in enumerate(cargas_divididas, start=0):
    proceso = pool.apply_async(procesar_nucleo, args=(carga, i))
    procesos.append(proceso)

# Obtener los resultados
resultados = [p.get() for p in procesos]
# Cerrar el pool
pool.close()
pool.join()
    

AttributeError: 'int' object has no attribute 'get'